In [1]:
import constti
import inputFPL1
import Brr_functions
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np
from pathlib import Path


def outputFPL(d1,team_number,bigTable,Fixtures,lastGW,Gameweeks,teams,players,teamplayers):
    
    team_number = 20
    url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
    url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
    url3 = "https://fantasy.premierleague.com/api/event/6/live/"
    url4 = "https://fantasy.premierleague.com/api/fixtures"

    #NaNs to zeros
    def toint(a):
        if np.isnan(a):
            return 0
        else: return int(a)

    #If no matches played not to devide by zero
    def noZ(a):
        b = a.copy()
        for i in range(len(b)):
            if b[i] == 0:
                b[i]=1
        return b

    #Kills unfinished matches
    def is_finished(n):
        if n=='':
            return False
        else:
            a = Fixtures[Fixtures['id']==n]['finished']
            #print(a)
            return a.bool()







    #Fixtures
    Team_fixtures = pd.DataFrame()
    for j in range(lastGW,0,-1): 
        Team_fixtures['GW'+str(j)] = [Fixtures[((Fixtures['team_a']==i)|(Fixtures['team_h']==i))&\
                                    (Fixtures['event']==j)]['id'].values for i in range(1, team_number+1)]

    Player_fixtures = pd.DataFrame()
    for j in range(lastGW,0,-1): 
        Player_fixtures['GW'+str(j)] = [Gameweeks[(Gameweeks['id']==i)&\
                                    (Gameweeks['gameweek']==j)]['fixture'].values for i in bigTable['id']]

    #Matches
    TeamMatches = pd.DataFrame()
    TeamMatches['Matches'] = [len(Fixtures[Fixtures['finished']&((Fixtures['team_a']==i)|(Fixtures['team_h']==i))]) \
                                  for i in range(1,team_number+1)]

    PlayerMatches = pd.DataFrame()
    PlayerMatches['id'] = bigTable['id']
    PlayerMatches['Team number'] = [bigTable[bigTable['id'] == i]['team'].sum() for i in players.keys()]
    PlayerMatches['Team'] = [teams[PlayerMatches.at[i,'Team number']] for i in range(len(players))]
    PlayerMatches['Team games'] = [TeamMatches.at[PlayerMatches.at[i,'Team number']-1,'Matches'] for i in PlayerMatches.index]
    PlayerMatches['Played'] = [len(Gameweeks[(Gameweeks['id']==i)&(Gameweeks['minutes']>0)]) \
                            for i in PlayerMatches['id']]


    #Team tables

    print(0)
    #1. Creating  a table with average threat and GW threats for teams

    TeamThreat = pd.DataFrame()
    TeamThreat['id'] = pd.DataFrame(d1['teams'])['id']
    TeamThreat['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamThreat['Threat av'] = np.zeros(len(TeamThreat))
    TeamThreat['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamThreat['Threat GW'+str(j)] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                TeamThreat.at[i,'Threat GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Team_fixtures.at[i, 'GW'+str(j)][k])&\
                                                               (Gameweeks['team']==i+1)]['threat'].sum())


    TeamThreat['Threat av'] = [Gameweeks[Gameweeks['team']==i]['threat'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamMatches['Matches'])

    print(1)
    #2. Creating  a table with average creativity and GW creativities for teams

    TeamCreativity = pd.DataFrame()
    TeamCreativity['id'] = pd.DataFrame(d1['teams'])['id']
    TeamCreativity['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamCreativity['Creativity av'] = np.zeros(len(TeamCreativity))
    TeamCreativity['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamCreativity['Creativity GW'+str(j)] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                TeamCreativity.at[i,'Creativity GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Team_fixtures.at[i, 'GW'+str(j)][k])&\
                                                               (Gameweeks['team']==i+1)]['creativity'].sum())


    TeamCreativity['Creativity av'] = [Gameweeks[Gameweeks['team']==i]['creativity'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamMatches['Matches'])
    print(2)
    #3. Creating  a table with average threat allowed by teams and GW threat allowed

    TableDefence = pd.DataFrame()
    TableDefence['id'] = pd.DataFrame(d1['teams'])['id']
    TableDefence['Team'] = pd.DataFrame(d1['teams'])['name']
    TableDefence['Threat allowed av'] = np.zeros(len(TableDefence))
    TableDefence['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TableDefence['Threat allowed GW'+str(j)] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                TableDefence.at[i,'Threat allowed GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']== \
                    Team_fixtures.at[i, 'GW'+str(j)][k])&(Gameweeks['teamAgainst']==i+1)]['threat'].sum())


    TableDefence['Threat allowed av'] = [Gameweeks[Gameweeks['teamAgainst']==i]['threat'].sum() for i in range(1,team_number+1)] \
            /noZ(TeamMatches['Matches'])

    threatAllowedAv = TableDefence['Threat allowed av'].mean()
    print(3)
    #4. Creating  a table with average adjusted threat and GW threats adj for teams

    TeamThreatAd = pd.DataFrame()
    TeamThreatAd['id'] = pd.DataFrame(d1['teams'])['id']
    TeamThreatAd['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamThreatAd['Threat av adj'] = np.zeros(len(TeamThreatAd))
    TeamThreatAd['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamThreatAd['Threat GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                    TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'].append(TeamThreat.at[i,'Threat GW'+str(j)][k]*threatAllowedAv/ \
                        TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                      TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'].append(TeamThreat.at[i,'Threat GW'+str(j)][k]*threatAllowedAv/\
                        TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                TeamThreatAd.at[i,'Threat av adj'] = TeamThreatAd.at[i,'Threat av adj']  + \
                    TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'][k]


    TeamThreatAd['Threat av adj'] = TeamThreatAd['Threat av adj']/noZ(TeamMatches['Matches'])
    print(4)
    #5. Creating  a table with average adjusted creativity and GW creativities adj for teams

    TeamCreativityAd = pd.DataFrame()
    TeamCreativityAd['id'] = pd.DataFrame(d1['teams'])['id']
    TeamCreativityAd['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamCreativityAd['Creativity av adj'] = np.zeros(len(TeamCreativityAd))
    TeamCreativityAd['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamCreativityAd['Creativity GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                    TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'].append(TeamCreativity.at[i,'Creativity GW'+str(j)][k]*\
                        threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                      TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'].append(TeamCreativity.at[i,'Creativity GW'+str(j)][k]*\
                        threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                TeamCreativityAd.at[i,'Creativity av adj'] = TeamCreativityAd.at[i,'Creativity av adj']  + \
                    TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'][k]


    TeamCreativityAd['Creativity av adj'] = TeamCreativityAd['Creativity av adj']/noZ(TeamMatches['Matches'])
    print(5)
    #6. Creating  a table with average threat allowed adjusted by teams and GW threat allowed adjusted

    TableDefenceAd = pd.DataFrame()
    TableDefenceAd['id'] = pd.DataFrame(d1['teams'])['id']
    TableDefenceAd['Team'] = pd.DataFrame(d1['teams'])['name']
    TableDefenceAd['Threat allowed av adj'] = np.zeros(len(TableDefenceAd))
    TableDefenceAd['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TableDefenceAd['Threat allowed GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                    TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'].append(TableDefence.at[i,'Threat allowed GW'+str(j)][k]*\
                        threatAllowedAv/TeamThreat.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat av'])
                else:
                    TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'].append(TableDefence.at[i,'Threat allowed GW'+str(j)][k]*\
                        threatAllowedAv/TeamThreat.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat av'])

                TableDefenceAd.at[i,'Threat allowed av adj'] = TableDefenceAd.at[i,'Threat allowed av adj']  + \
                    TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'][k]


    TableDefenceAd['Threat allowed av adj'] = TableDefenceAd['Threat allowed av adj']/noZ(TeamMatches['Matches'])

    print(6)


    #Total Team Table

    TableTeams = pd.DataFrame()
    TableTeams['id'] = pd.DataFrame(d1['teams'])['id']
    TableTeams['Team'] = pd.DataFrame(d1['teams'])['name']

    TableTeams['Threat adjusted'] = TeamThreatAd['Threat av adj']
    TableTeams['Threat'] = TeamThreat['Threat av']
    TableTeams['Creativity adjusted'] = TeamCreativityAd['Creativity av adj']
    TableTeams['Creativity'] = TeamCreativity['Creativity av']
    TableTeams['Threat allowed adjusted'] = TableDefenceAd['Threat allowed av adj']
    TableTeams['Threat allowed'] = TableDefence['Threat allowed av']
    print(7)



    #PLayer Tables


    #1 Players Threat

    PlayerThreat = pd.DataFrame()
    PlayerThreat['id'] = bigTable['id']
    PlayerThreat['Name'] = bigTable['full_name']
    PlayerThreat['Team'] = PlayerMatches['Team']
    PlayerThreat['Threat per fixture'] = np.zeros(len(players))
    PlayerThreat['Threat per game'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerThreat['Threat GW'+str(j)] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
                PlayerThreat.at[i,'Threat GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Player_fixtures.at[i, 'GW'+str(j)][k])&\
                                                               (Gameweeks['id']==PlayerThreat.at[i,'id'])]['threat'].sum())
                PlayerThreat.at[i,'Threat per game'] = PlayerThreat.at[i,'Threat per game'] +\
                    PlayerThreat.at[i,'Threat GW'+str(j)][k]


    PlayerThreat['Threat per fixture'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Team games'])
    PlayerThreat['Threat per game'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Played'])
    print(9)
    #2 Players Creativity

    PlayerCreativity = pd.DataFrame()
    PlayerCreativity['id'] = bigTable['id']
    PlayerCreativity['Name'] = bigTable['full_name']
    PlayerCreativity['Team'] = PlayerMatches['Team']
    PlayerCreativity['Creativity per fixture'] = np.zeros(len(players))
    PlayerCreativity['Creativity per game'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerCreativity['Creativity GW'+str(j)] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
                PlayerCreativity.at[i,'Creativity GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==\
                                            Player_fixtures.at[i, 'GW'+str(j)][k])&\
                                            (Gameweeks['id']==PlayerCreativity.at[i,'id'])]['creativity'].sum())
                PlayerCreativity.at[i,'Creativity per game'] = PlayerCreativity.at[i,'Creativity per game'] +\
                    PlayerCreativity.at[i,'Creativity GW'+str(j)][k]


    PlayerCreativity['Creativity per fixture'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Team games'])
    PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Played'])
    print(10)
    #3 Players Threat Adjusted

    PlayerThreatAd = pd.DataFrame()
    PlayerThreatAd['id'] = bigTable['id']
    PlayerThreatAd['Name'] = bigTable['full_name']
    PlayerThreatAd['Team number'] = PlayerMatches['Team number']
    PlayerThreatAd['Team'] = PlayerMatches['Team']
    PlayerThreatAd['Threat per fixture adj'] = np.zeros(len(players))
    PlayerThreatAd['Threat per game adj'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerThreatAd['Threat GW'+str(j) + 'adj'] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):

                if toint(Fixtures[Fixtures['id']==Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==\
                    PlayerThreatAd.at[i,'Team number']:
                    PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'].append(PlayerThreat.at[i,'Threat GW'+str(j)][k]\
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                      PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'].append(PlayerThreat.at[i,'Threat GW'+str(j)][k]
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                PlayerThreatAd.at[i,'Threat per game adj'] = PlayerThreatAd.at[i,'Threat per game adj'] +\
                    PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'][k]


    PlayerThreatAd['Threat per fixture adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Team games'])
    PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Played'])
    print(11)

    #4 PLayers Creativity Adjusted

    PlayerCreativityAd = pd.DataFrame()
    PlayerCreativityAd['id'] = bigTable['id']
    PlayerCreativityAd['Name'] = bigTable['full_name']
    PlayerCreativityAd['Team number'] = PlayerMatches['Team number']
    PlayerCreativityAd['Team'] = PlayerMatches['Team']
    PlayerCreativityAd['Creativity per fixture adj'] = np.zeros(len(players))
    PlayerCreativityAd['Creativity per game adj'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerCreativityAd['Creativity GW'+str(j) + 'adj'] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):

                if toint(Fixtures[Fixtures['id']==Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==\
                    PlayerCreativityAd.at[i,'Team number']:
                    PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'].append(PlayerCreativity.at[i,'Creativity GW'+str(j)][k]\
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                    PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'].append(PlayerCreativity.at[i,'Creativity GW'+str(j)][k]\
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                PlayerCreativityAd.at[i,'Creativity per game adj'] = PlayerCreativityAd.at[i,'Creativity per game adj'] +\
                    PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'][k]


    PlayerCreativityAd['Creativity per fixture adj'] = PlayerCreativityAd['Creativity per game adj']/\
    noZ(PlayerMatches['Team games'])
    PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj']/noZ(PlayerMatches['Played'])
    print(12)


    #Tables2Files

    del TeamThreat['id']
    TeamThreat.sort_values('Threat av', ascending = False, inplace = True)
    TeamThreat.index = np.arange(1, len(TeamThreat) + 1)
    TeamThreat = Brr_functions.no_lists(TeamThreat)
    TeamThreat.to_csv(Path('out/TeamThreat.csv'))

    del TeamCreativity['id']
    TeamCreativity.sort_values('Creativity av', ascending = False, inplace = True)
    TeamCreativity.index = np.arange(1, len(TeamCreativity) + 1)
    TeamCreativity = Brr_functions.no_lists(TeamCreativity)
    TeamCreativity.to_csv(Path('out/TeamCreativity.csv'))

    del TableDefence['id']
    TableDefence.sort_values('Threat allowed av', ascending = True, inplace = True)
    TableDefence.index = np.arange(1, len(TableDefence) + 1)
    TableDefence = Brr_functions.no_lists(TableDefence)
    TableDefence.to_csv(Path('out/TableDefence.csv'))

    del TeamThreatAd['id']
    TeamThreatAd.sort_values('Threat av adj', ascending = False, inplace = True)
    TeamThreatAd.index = np.arange(1, len(TeamThreatAd) + 1)
    TeamThreatAd = Brr_functions.no_lists(TeamThreatAd)
    TeamThreatAd.to_csv(Path('out/TeamThreatAd.csv'))

    del TeamCreativityAd['id']
    TeamCreativityAd.sort_values('Creativity av adj', ascending = False, inplace = True)
    TeamCreativityAd.index = np.arange(1, len(TeamCreativityAd) + 1)
    TeamCreativityAd = Brr_functions.no_lists(TeamCreativityAd)
    TeamCreativityAd.to_csv(Path('out/TeamCreativityAd.csv'))

    del TableDefenceAd['id']
    TableDefenceAd.sort_values('Threat allowed av adj', ascending = True, inplace = True)
    TableDefenceAd.index = np.arange(1, len(TableDefenceAd) + 1)
    TableDefenceAd = Brr_functions.no_lists(TableDefenceAd)
    TableDefenceAd.to_csv(Path('out/TableDefenceAd.csv'))

    del TableTeams['id']
    TableTeams.sort_values('Threat adjusted', ascending = False, inplace = True)
    TableTeams.index = np.arange(1, len(TableTeams) + 1)
    TableTeams.to_csv(Path('out/TableTeams.csv'))

    del PlayerThreat['id']
    PlayerThreat.sort_values('Threat per fixture', ascending = False, inplace = True)
    PlayerThreat.index = np.arange(1, len(players) + 1)
    PlayerThreat = Brr_functions.no_lists(PlayerThreat)
    PlayerThreat.to_csv(Path('out/PlayerThreat.csv'))

    del PlayerCreativity['id']
    PlayerCreativity.sort_values('Creativity per fixture', ascending = False, inplace = True)
    PlayerCreativity.index = np.arange(1, len(players) + 1)
    PlayerCreativity = Brr_functions.no_lists(PlayerCreativity)
    PlayerCreativity.to_csv(Path('out/PlayerCreativity.csv'))

    del PlayerThreatAd['id']
    del PlayerThreatAd['Team number']
    PlayerThreatAd.sort_values('Threat per fixture adj', ascending = False, inplace = True)
    PlayerThreatAd.index = np.arange(1, len(players) + 1)
    PlayerThreatAd = Brr_functions.no_lists(PlayerThreatAd)
    PlayerThreatAd.to_csv(Path('out/PlayerThreatAd.csv'))

    del PlayerCreativityAd['id']
    del PlayerCreativityAd['Team number']
    PlayerCreativityAd.sort_values('Creativity per fixture adj', ascending = False, inplace = True)
    PlayerCreativityAd.index = np.arange(1, len(players) + 1)
    PlayerCreativityAd = Brr_functions.no_lists(PlayerCreativityAd)
    PlayerCreativityAd.to_csv(Path('out/PlayerCreativityAd.csv'))
    
    return TeamThreat, TeamCreativity, TableDefence, TeamThreatAd, TeamCreativityAd, TableDefenceAd, \
            TableTeams, PlayerThreat, PlayerCreativity, PlayerThreatAd, PlayerCreativityAd


if __name__=='__main__':
    
    d1,team_number,bigTable,Fixtures,lastGW,Gameweeks,teams,players,teamplayers = inputFPL1.inputFPL()
    
    TeamThreat, TeamCreativity, TableDefence, TeamThreatAd, TeamCreativityAd, TableDefenceAd, \
    TableTeams, PlayerThreat, PlayerCreativity, PlayerThreatAd, PlayerCreativityAd \
    = outputFPL(d1,team_number,bigTable,Fixtures,lastGW,Gameweeks,teams,players,teamplayers)
    
    display(TeamThreat)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
9
10
11
12


,Team,Threat av,Matches,Threat GW24,Threat GW24*,Threat GW23,Threat GW22,Threat GW21,Threat GW20,Threat GW19,...,Threat GW10,Threat GW9,Threat GW8,Threat GW7,Threat GW6,Threat GW5,Threat GW4,Threat GW3,Threat GW2,Threat GW1
1,Man City,295.875000,24,238,,407,312,184,184,84,...,435,309,240,257,401,382,185,317,469,237
2,Chelsea,227.916667,24,320,,240,299,252,176,128,...,245,259,157,339,103,229,162,287,163,160
3,Liverpool,224.304348,23,202,0,205,255,292,142,211,...,343,117,258,151,122,318,202,362,163,222
4,Leicester,190.250000,24,299,,243,123,234,219,29,...,392,173,46,187,196,112,181,130,113,130
5,Everton,181.083333,24,160,,177,239,112,320,236,...,143,300,145,235,216,209,230,131,119,126
6,Man Utd,180.916667,24,254,,90,231,145,201,224,...,284,60,121,162,126,91,276,231,110,190
7,Wolves,172.541667,24,126,,126,158,159,81,294,...,165,45,73,88,214,170,184,180,84,90
8,Brighton,167.083333,24,301,,180,113,224,179,89,...,94,238,242,92,207,181,111,180,167,71
9,Arsenal,162.000000,24,54,,142,151,174,80,244,...,189,127,187,145,262,102,304,88,226,149
10,Southampton,159.250000,24,149,,124,198,163,211,89,...,68,174,107,175,264,152,126,174,178,146


In [1]:
import constti
import inputFPL1
import Brr_functions
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np
from pathlib import Path

d1,team_number,bigTable,Fixtures,lastGW,Gameweeks,teams,players,teamplayers = inputFPL1.inputFPL()

if 1: 
    team_number = 20
    url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
    url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
    url3 = "https://fantasy.premierleague.com/api/event/6/live/"
    url4 = "https://fantasy.premierleague.com/api/fixtures"

    #NaNs to zeros
    def toint(a):
        if np.isnan(a):
            return 0
        else: return int(a)

    #If no matches played not to devide by zero
    def noZ(a):
        b = a.copy()
        for i in range(len(b)):
            if b[i] == 0:
                b[i]=1
        return b

    #Kills unfinished matches
    def is_finished(n):
        if n=='':
            return False
        else:
            a = Fixtures[Fixtures['id']==n]['finished']
            #print(a)
            return a.bool()







    #Fixtures
    Team_fixtures = pd.DataFrame()
    for j in range(lastGW,0,-1): 
        Team_fixtures['GW'+str(j)] = [Fixtures[((Fixtures['team_a']==i)|(Fixtures['team_h']==i))&\
                                    (Fixtures['event']==j)]['id'].values for i in range(1, team_number+1)]

    Player_fixtures = pd.DataFrame()
    for j in range(lastGW,0,-1): 
        Player_fixtures['GW'+str(j)] = [Gameweeks[(Gameweeks['id']==i)&\
                                    (Gameweeks['gameweek']==j)]['fixture'].values for i in bigTable['id']]

    #Matches
    TeamMatches = pd.DataFrame()
    TeamMatches['Matches'] = [len(Fixtures[Fixtures['finished']&((Fixtures['team_a']==i)|(Fixtures['team_h']==i))]) \
                                  for i in range(1,team_number+1)]

    PlayerMatches = pd.DataFrame()
    PlayerMatches['id'] = bigTable['id']
    PlayerMatches['Team number'] = [bigTable[bigTable['id'] == i]['team'].sum() for i in players.keys()]
    PlayerMatches['Team'] = [teams[PlayerMatches.at[i,'Team number']] for i in range(len(players))]
    PlayerMatches['Team games'] = [TeamMatches.at[PlayerMatches.at[i,'Team number']-1,'Matches'] for i in PlayerMatches.index]
    PlayerMatches['Played'] = [len(Gameweeks[(Gameweeks['id']==i)&(Gameweeks['minutes']>0)]) \
                            for i in PlayerMatches['id']]


    #Team tables

    print(0)
    #1. Creating  a table with average threat and GW threats for teams

    TeamThreat = pd.DataFrame()
    TeamThreat['id'] = pd.DataFrame(d1['teams'])['id']
    TeamThreat['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamThreat['Threat av'] = np.zeros(len(TeamThreat))
    TeamThreat['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamThreat['Threat GW'+str(j)] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                TeamThreat.at[i,'Threat GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Team_fixtures.at[i, 'GW'+str(j)][k])&\
                                                               (Gameweeks['team']==i+1)]['threat'].sum())


    TeamThreat['Threat av'] = [Gameweeks[Gameweeks['team']==i]['threat'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamMatches['Matches'])

    print(1)
    #2. Creating  a table with average creativity and GW creativities for teams

    TeamCreativity = pd.DataFrame()
    TeamCreativity['id'] = pd.DataFrame(d1['teams'])['id']
    TeamCreativity['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamCreativity['Creativity av'] = np.zeros(len(TeamCreativity))
    TeamCreativity['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamCreativity['Creativity GW'+str(j)] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                TeamCreativity.at[i,'Creativity GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Team_fixtures.at[i, 'GW'+str(j)][k])&\
                                                               (Gameweeks['team']==i+1)]['creativity'].sum())


    TeamCreativity['Creativity av'] = [Gameweeks[Gameweeks['team']==i]['creativity'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamMatches['Matches'])
    print(2)
    #3. Creating  a table with average threat allowed by teams and GW threat allowed

    TableDefence = pd.DataFrame()
    TableDefence['id'] = pd.DataFrame(d1['teams'])['id']
    TableDefence['Team'] = pd.DataFrame(d1['teams'])['name']
    TableDefence['Threat allowed av'] = np.zeros(len(TableDefence))
    TableDefence['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TableDefence['Threat allowed GW'+str(j)] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                TableDefence.at[i,'Threat allowed GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']== \
                    Team_fixtures.at[i, 'GW'+str(j)][k])&(Gameweeks['teamAgainst']==i+1)]['threat'].sum())


    TableDefence['Threat allowed av'] = [Gameweeks[Gameweeks['teamAgainst']==i]['threat'].sum() for i in range(1,team_number+1)] \
            /noZ(TeamMatches['Matches'])

    threatAllowedAv = TableDefence['Threat allowed av'].mean()
    print(3)
    #4. Creating  a table with average adjusted threat and GW threats adj for teams

    TeamThreatAd = pd.DataFrame()
    TeamThreatAd['id'] = pd.DataFrame(d1['teams'])['id']
    TeamThreatAd['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamThreatAd['Threat av adj'] = np.zeros(len(TeamThreatAd))
    TeamThreatAd['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamThreatAd['Threat GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                    TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'].append(TeamThreat.at[i,'Threat GW'+str(j)][k]*threatAllowedAv/ \
                        TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                      TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'].append(TeamThreat.at[i,'Threat GW'+str(j)][k]*threatAllowedAv/\
                        TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                TeamThreatAd.at[i,'Threat av adj'] = TeamThreatAd.at[i,'Threat av adj']  + \
                    TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'][k]


    TeamThreatAd['Threat av adj'] = TeamThreatAd['Threat av adj']/noZ(TeamMatches['Matches'])
    print(4)
    #5. Creating  a table with average adjusted creativity and GW creativities adj for teams

    TeamCreativityAd = pd.DataFrame()
    TeamCreativityAd['id'] = pd.DataFrame(d1['teams'])['id']
    TeamCreativityAd['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamCreativityAd['Creativity av adj'] = np.zeros(len(TeamCreativityAd))
    TeamCreativityAd['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamCreativityAd['Creativity GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                    TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'].append(TeamCreativity.at[i,'Creativity GW'+str(j)][k]*\
                        threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                      TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'].append(TeamCreativity.at[i,'Creativity GW'+str(j)][k]*\
                        threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                TeamCreativityAd.at[i,'Creativity av adj'] = TeamCreativityAd.at[i,'Creativity av adj']  + \
                    TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'][k]


    TeamCreativityAd['Creativity av adj'] = TeamCreativityAd['Creativity av adj']/noZ(TeamMatches['Matches'])
    print(5)
    #6. Creating  a table with average threat allowed adjusted by teams and GW threat allowed adjusted

    TableDefenceAd = pd.DataFrame()
    TableDefenceAd['id'] = pd.DataFrame(d1['teams'])['id']
    TableDefenceAd['Team'] = pd.DataFrame(d1['teams'])['name']
    TableDefenceAd['Threat allowed av adj'] = np.zeros(len(TableDefenceAd))
    TableDefenceAd['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TableDefenceAd['Threat allowed GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                    TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'].append(TableDefence.at[i,'Threat allowed GW'+str(j)][k]*\
                        threatAllowedAv/TeamThreat.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat av'])
                else:
                    TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'].append(TableDefence.at[i,'Threat allowed GW'+str(j)][k]*\
                        threatAllowedAv/TeamThreat.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat av'])

                TableDefenceAd.at[i,'Threat allowed av adj'] = TableDefenceAd.at[i,'Threat allowed av adj']  + \
                    TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'][k]


    TableDefenceAd['Threat allowed av adj'] = TableDefenceAd['Threat allowed av adj']/noZ(TeamMatches['Matches'])

    print(6)


    #Total Team Table

    TableTeams = pd.DataFrame()
    TableTeams['id'] = pd.DataFrame(d1['teams'])['id']
    TableTeams['Team'] = pd.DataFrame(d1['teams'])['name']

    TableTeams['Threat adjusted'] = TeamThreatAd['Threat av adj']
    TableTeams['Threat'] = TeamThreat['Threat av']
    TableTeams['Creativity adjusted'] = TeamCreativityAd['Creativity av adj']
    TableTeams['Creativity'] = TeamCreativity['Creativity av']
    TableTeams['Threat allowed adjusted'] = TableDefenceAd['Threat allowed av adj']
    TableTeams['Threat allowed'] = TableDefence['Threat allowed av']
    print(7)



    #PLayer Tables


    #1 Players Threat

    PlayerThreat = pd.DataFrame()
    PlayerThreat['id'] = bigTable['id']
    PlayerThreat['Name'] = bigTable['full_name']
    PlayerThreat['Team'] = PlayerMatches['Team']
    PlayerThreat['Threat per fixture'] = np.zeros(len(players))
    PlayerThreat['Threat per game'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerThreat['Threat GW'+str(j)] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
                PlayerThreat.at[i,'Threat GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Player_fixtures.at[i, 'GW'+str(j)][k])&\
                                                               (Gameweeks['id']==PlayerThreat.at[i,'id'])]['threat'].sum())
                PlayerThreat.at[i,'Threat per game'] = PlayerThreat.at[i,'Threat per game'] +\
                    PlayerThreat.at[i,'Threat GW'+str(j)][k]


    PlayerThreat['Threat per fixture'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Team games'])
    PlayerThreat['Threat per game'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Played'])
    print(9)
    #2 Players Creativity

    PlayerCreativity = pd.DataFrame()
    PlayerCreativity['id'] = bigTable['id']
    PlayerCreativity['Name'] = bigTable['full_name']
    PlayerCreativity['Team'] = PlayerMatches['Team']
    PlayerCreativity['Creativity per fixture'] = np.zeros(len(players))
    PlayerCreativity['Creativity per game'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerCreativity['Creativity GW'+str(j)] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
                PlayerCreativity.at[i,'Creativity GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==\
                                            Player_fixtures.at[i, 'GW'+str(j)][k])&\
                                            (Gameweeks['id']==PlayerCreativity.at[i,'id'])]['creativity'].sum())
                PlayerCreativity.at[i,'Creativity per game'] = PlayerCreativity.at[i,'Creativity per game'] +\
                    PlayerCreativity.at[i,'Creativity GW'+str(j)][k]


    PlayerCreativity['Creativity per fixture'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Team games'])
    PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Played'])
    print(10)
    #3 Players Threat Adjusted

    PlayerThreatAd = pd.DataFrame()
    PlayerThreatAd['id'] = bigTable['id']
    PlayerThreatAd['Name'] = bigTable['full_name']
    PlayerThreatAd['Team number'] = PlayerMatches['Team number']
    PlayerThreatAd['Team'] = PlayerMatches['Team']
    PlayerThreatAd['Threat per fixture adj'] = np.zeros(len(players))
    PlayerThreatAd['Threat per game adj'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerThreatAd['Threat GW'+str(j) + 'adj'] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
                #If player's team is away
                if toint(Fixtures[Fixtures['id']==Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==\
                    PlayerThreatAd.at[i,'Team number']:
                    PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'].append(PlayerThreat.at[i,'Threat GW'+str(j)][k]\
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                #If home
                else:
                      PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'].append(PlayerThreat.at[i,'Threat GW'+str(j)][k]
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                PlayerThreatAd.at[i,'Threat per game adj'] = PlayerThreatAd.at[i,'Threat per game adj'] +\
                    PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'][k]


    PlayerThreatAd['Threat per fixture adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Team games'])
    PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Played'])
    
    display(PlayerThreatAd.sort_values('Threat per fixture adj', ascending = False))
    print(11)

    #4 PLayers Creativity Adjusted

    PlayerCreativityAd = pd.DataFrame()
    PlayerCreativityAd['id'] = bigTable['id']
    PlayerCreativityAd['Name'] = bigTable['full_name']
    PlayerCreativityAd['Team number'] = PlayerMatches['Team number']
    PlayerCreativityAd['Team'] = PlayerMatches['Team']
    PlayerCreativityAd['Creativity per fixture adj'] = np.zeros(len(players))
    PlayerCreativityAd['Creativity per game adj'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerCreativityAd['Creativity GW'+str(j) + 'adj'] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):

                if toint(Fixtures[Fixtures['id']==Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==\
                    PlayerCreativityAd.at[i,'Team number']:
                    PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'].append(PlayerCreativity.at[i,'Creativity GW'+str(j)][k]\
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                    PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'].append(PlayerCreativity.at[i,'Creativity GW'+str(j)][k]\
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                PlayerCreativityAd.at[i,'Creativity per game adj'] = PlayerCreativityAd.at[i,'Creativity per game adj'] +\
                    PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'][k]


    PlayerCreativityAd['Creativity per fixture adj'] = PlayerCreativityAd['Creativity per game adj']/\
    noZ(PlayerMatches['Team games'])
    PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj']/noZ(PlayerMatches['Played'])
    print(12)


    #Tables2Files

    del TeamThreat['id']
    TeamThreat.sort_values('Threat av', ascending = False, inplace = True)
    TeamThreat.index = np.arange(1, len(TeamThreat) + 1)
    TeamThreat = Brr_functions.no_lists(TeamThreat)
    TeamThreat.to_csv(Path('out/TeamThreat.csv'))

    del TeamCreativity['id']
    TeamCreativity.sort_values('Creativity av', ascending = False, inplace = True)
    TeamCreativity.index = np.arange(1, len(TeamCreativity) + 1)
    TeamCreativity = Brr_functions.no_lists(TeamCreativity)
    TeamCreativity.to_csv(Path('out/TeamCreativity.csv'))

    del TableDefence['id']
    TableDefence.sort_values('Threat allowed av', ascending = True, inplace = True)
    TableDefence.index = np.arange(1, len(TableDefence) + 1)
    TableDefence = Brr_functions.no_lists(TableDefence)
    TableDefence.to_csv(Path('out/TableDefence.csv'))

    del TeamThreatAd['id']
    TeamThreatAd.sort_values('Threat av adj', ascending = False, inplace = True)
    TeamThreatAd.index = np.arange(1, len(TeamThreatAd) + 1)
    TeamThreatAd = Brr_functions.no_lists(TeamThreatAd)
    TeamThreatAd.to_csv(Path('out/TeamThreatAd.csv'))

    del TeamCreativityAd['id']
    TeamCreativityAd.sort_values('Creativity av adj', ascending = False, inplace = True)
    TeamCreativityAd.index = np.arange(1, len(TeamCreativityAd) + 1)
    TeamCreativityAd = Brr_functions.no_lists(TeamCreativityAd)
    TeamCreativityAd.to_csv(Path('out/TeamCreativityAd.csv'))

    del TableDefenceAd['id']
    TableDefenceAd.sort_values('Threat allowed av adj', ascending = True, inplace = True)
    TableDefenceAd.index = np.arange(1, len(TableDefenceAd) + 1)
    TableDefenceAd = Brr_functions.no_lists(TableDefenceAd)
    TableDefenceAd.to_csv(Path('out/TableDefenceAd.csv'))

    del TableTeams['id']
    TableTeams.sort_values('Threat adjusted', ascending = False, inplace = True)
    TableTeams.index = np.arange(1, len(TableTeams) + 1)
    TableTeams.to_csv(Path('out/TableTeams.csv'))

    del PlayerThreat['id']
    PlayerThreat.sort_values('Threat per fixture', ascending = False, inplace = True)
    PlayerThreat.index = np.arange(1, len(players) + 1)
    PlayerThreat = Brr_functions.no_lists(PlayerThreat)
    PlayerThreat.to_csv(Path('out/PlayerThreat.csv'))

    del PlayerCreativity['id']
    PlayerCreativity.sort_values('Creativity per fixture', ascending = False, inplace = True)
    PlayerCreativity.index = np.arange(1, len(players) + 1)
    PlayerCreativity = Brr_functions.no_lists(PlayerCreativity)
    PlayerCreativity.to_csv(Path('out/PlayerCreativity.csv'))

    del PlayerThreatAd['id']
    del PlayerThreatAd['Team number']
    PlayerThreatAd.sort_values('Threat per fixture adj', ascending = False, inplace = True)
    PlayerThreatAd.index = np.arange(1, len(players) + 1)
    PlayerThreatAd = Brr_functions.no_lists(PlayerThreatAd)
    PlayerThreatAd.to_csv(Path('out/PlayerThreatAd.csv'))

    del PlayerCreativityAd['id']
    del PlayerCreativityAd['Team number']
    PlayerCreativityAd.sort_values('Creativity per fixture adj', ascending = False, inplace = True)
    PlayerCreativityAd.index = np.arange(1, len(players) + 1)
    PlayerCreativityAd = Brr_functions.no_lists(PlayerCreativityAd)
    PlayerCreativityAd.to_csv(Path('out/PlayerCreativityAd.csv'))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
9
10


,id,Name,Team number,Team,Threat per fixture adj,Threat per game adj,Threat GW27adj,Threat GW26adj,Threat GW25adj,Threat GW24adj,...,Threat GW10adj,Threat GW9adj,Threat GW8adj,Threat GW7adj,Threat GW6adj,Threat GW5adj,Threat GW4adj,Threat GW3adj,Threat GW2adj,Threat GW1adj
283,191,Mohamed Salah,10,Liverpool,60.678587,68.593186,[],[62.84208194545645],[98.59621998809446],"[115.28120013273892, 151.01161784161786]",...,[84.16786549086274],[0.0],[109.84699069083783],[81.94832855006266],[49.94587463353344],[58.3935854954254],[57.09988772764048],[60.303893702451404],[28.592903796547393],[30.257298714479028]
279,187,Roberto Firmino,10,Liverpool,54.500504,54.500504,[],[27.15398602581451],[65.07350519214233],"[89.47197622242423, 117.20304668304671]",...,[102.26848172545688],[53.06721818807016],[47.911985301322886],[12.05122478677392],[80.46835357624832],[23.197451772155297],[70.88261924810541],[89.10575338123417],[31.550790396190227],[51.20465936296451]
601,409,Raúl Jiménez,20,Wolves,50.640305,50.640305,[38.015580436140326],[49.08057030867222],[45.8307793442424],[83.8047083626142],...,[10.398857690966167],[22.677130597261726],[48.416919287608955],[12.243747433264888],[25.424101037537707],[18.03601028433152],[63.736899718912085],[42.33267538428518],[26.53360909403508],[35.05755022048016]
314,214,Raheem Sterling,11,Man City,47.778340,56.087616,[0.0],[0.0],[25.340862728431794],[65.07661384857917],...,[56.6764598435657],[33.898801383383606],[55.22066622566624],[89.70378478957996],[0.0],[73.70367635578225],[9.934364639043002],[53.115134509175135],[110.41375903102424],[63.662752312109546]
468,313,Danny Ings,16,Southampton,42.507551,42.507551,[30.7282011200055],[20.674097280697413],[156.7458434189636],[0.0],...,[1.1685850073493387],[92.41009450009452],[84.63050979570944],[40.72638652783681],[35.95486028313394],[2.4102449573547844],[28.945755375310995],[12.914674030755904],[91.56440358137478],[0.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,578,Sam Woods,7,Crystal Palace,0.000000,0.000000,[0.0],[0.0],[0.0],[0.0],...,[],[],[],[],[],[],[],[],[],[]
209,556,Stephen Henderson,7,Crystal Palace,0.000000,0.000000,[0.0],[0.0],[0.0],[0.0],...,[0.0],[0.0],[],[],[],[],[],[],[],[]
471,316,Sam Gallagher,16,Southampton,0.000000,0.000000,[0.0],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
472,317,Fraser Forster,16,Southampton,0.000000,0.000000,[0.0],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]


11
12


In [2]:
Team_fixtures

,GW26,GW25,GW24,GW23,GW22,GW21,GW20,GW19,GW18,GW17,...,GW10,GW9,GW8,GW7,GW6,GW5,GW4,GW3,GW2,GW1
0,[251],[242],[238],[221],[214],[201],[191],[182],[174],[161],...,[91],[88],[71],[67],[51],[49],[31],[24],[11],[10]
1,[252],[241],[231],[222],[211],[203],[199],[181],[171],[168],...,[95],[81],[77],[61],[51],[41],[34],[21],[12],[6]
2,[258],[241],[232],[227],[212],[210],[192],[182],[172],[163],...,[99],[82],[71],[62],[59],[42],[36],[22],[12],[2]
3,[253],[250],[232],[222],[215],[202],[192],[189],[173],[164],...,[92],[81],[72],[63],[58],[43],[37],[23],[13],[7]
4,[259],[242],[235],[223],[213],[203],[193],[185],[172],[162],...,[93],[86],[73],[61],[52],[43],[32],[30],[11],[3]
5,[254],[244],[238],[226],[213],[202],[191],[183],[178],[163],...,[93],[83],[78],[63],[53],[50],[33],[26],[14],[9]
6,[255],[243],[239],[225],[214],[207],[198],[184],[176],[164],...,[91],[84],[80],[64],[54],[48],[34],[25],[18],[4]
7,[255],[249],[233],[230],[215],[205],[196],[185],[174],[167],...,[92],[85],[73],[65],[55],[42],[35],[21],[15],[4]
8,[260],[244],[234],[223],[216],[206],[200],[186],[175],[165],...,[98],[86],[74],[66],[56],[45],[36],[27],[14],[5]
9,[257],[245],"[237, 180]",[224],[219],[204],[194],[186],[],[166],...,[94],[87],[74],[68],[53],[44],[32],[24],[19],[1]


In [6]:
PlayerThreatAd

,Name,Team,Threat per fixture adj,Threat per game adj,Threat GW26adj,Threat GW25adj,Threat GW24adj*,Threat GW24adj,Threat GW23adj,Threat GW22adj,...,Threat GW10adj,Threat GW9adj,Threat GW8adj,Threat GW7adj,Threat GW6adj,Threat GW5adj,Threat GW4adj,Threat GW3adj,Threat GW2adj,Threat GW1adj
1,Mohamed Salah,Liverpool,60.591666,68.854166,,95.2213,148.963,118.046,67.5383,27.6014,...,85.5643,0,113.172,81.6504,49.0243,58.5098,56.288,59.6663,27.6142,30.5273
2,Roberto Firmino,Liverpool,55.632115,55.632115,,62.8461,115.613,91.6178,56.4866,75.4438,...,103.965,54.0307,49.3624,12.0074,78.9836,23.2436,69.8748,88.1636,30.4708,51.6616
3,Raheem Sterling,Man City,51.615767,56.104094,,25.7613,,64.84,47.9595,0,...,57.7923,33.2027,54.4715,89.2909,0,74.3614,10.0815,53.0999,112.246,65.1896
4,Raúl Jiménez,Wolves,50.768243,50.768243,,46.6628,,82.0579,76.1771,53.7008,...,10.4196,21.9009,46.7241,12.3827,24.902,17.7032,63.4436,41.7308,27.0153,36.1188
5,Danny Ings,Southampton,43.717172,43.717172,,153.479,,0,24.4566,124.008,...,1.20396,91.1563,83.069,41.4021,35.9445,2.40148,29.4713,13.106,89.6558,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,Wesley Hoedt,Southampton,0.000000,0.000000,,0,,0,0,0,...,0,0,0,0,0,0,0,0,0,0
620,Ruben Loftus-Cheek,Chelsea,0.000000,0.000000,,0,,0,0,0,...,0,0,0,0,0,0,0,0,0,0
621,Alexis Sánchez,Man Utd,0.000000,0.000000,,0,,0,0,0,...,0,0,0,0,0,0,0,0,0,0
622,Daniel N'Lundulu,Southampton,0.000000,0.000000,,0,,0,0,0,...,0,0,0,0,0,0,0,,,


In [10]:
Test.sort_values('Threat per fixture adj', ascending = False)

,id,Name,Team number,Team,Threat per fixture adj,Threat per game adj,Threat GW26adj,Threat GW25adj,Threat GW24adj,Threat GW23adj,...,Threat GW10adj,Threat GW9adj,Threat GW8adj,Threat GW7adj,Threat GW6adj,Threat GW5adj,Threat GW4adj,Threat GW3adj,Threat GW2adj,Threat GW1adj
282,191,Mohamed Salah,10,Liverpool,60.591666,68.854166,[],[95.22131331241596],"[118.04599834128136, 148.96280743066455]",[67.53833092485549],...,[85.56432979644867],[0.0],[113.17224426183056],[81.65037173913042],[49.02432692307692],[58.509823618185244],[56.288019643672904],[59.66630161391741],[27.614180860600626],[30.52730747973471]
278,187,Roberto Firmino,10,Liverpool,55.632115,55.632115,[],[62.846066786194534],"[91.6177897574124, 115.61292517006802]",[56.486604046242775],...,[103.9652609354699],[54.030664739884394],[49.36236185888354],[12.007407608695651],[78.98363782051281],[23.243628560648933],[69.87478300593878],[88.16363969817647],[30.470820259973106],[51.6615972733972]
313,214,Raheem Sterling,11,Man City,51.615767,56.104094,[],[25.76130359462971],[64.84000108695652],[47.959455646002915],...,[57.79225254015076],[33.2027000626174],[54.47147435897436],[89.29092688114862],[0.0],[74.36139001473839],[10.08151038101757],[53.0998687372897],[112.24567994802946],[65.18958117354344]
600,409,Raúl Jiménez,20,Wolves,50.768243,50.768243,[],[46.66284682080924],[82.05788626609441],[76.17705064993277],...,[10.419557630635728],[21.90090206185567],[46.72406524926686],[12.382719982754903],[24.902025046963054],[17.703229166666667],[63.44355331028981],[41.730773184102326],[27.015332369942197],[36.118801360158685]
467,313,Danny Ings,16,Southampton,43.717172,43.717172,[],[153.47863912732473],[0.0],[24.456580324437468],...,[1.2039600453386228],[91.15634484563056],[83.0689983974359],[41.40209506279775],[35.94452652985764],[2.40148152173913],[29.471271676300574],[13.105963495322838],[89.6558386981402],[0.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,485,Wesley Hoedt,16,Southampton,0.000000,0.000000,[],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
167,115,Ruben Loftus-Cheek,6,Chelsea,0.000000,0.000000,[],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
346,240,Alexis Sánchez,12,Man Utd,0.000000,0.000000,[],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
489,535,Daniel N'Lundulu,16,Southampton,0.000000,0.000000,[],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[],[],[]
